In [36]:
# Import Dependencies
import pandas as pd
import numpy as np
import os


In [37]:

#Files to load
census_data_to_load = os.path.join("..","Resources","Data","demographic_data.csv")
gini_file_to_load = os.path.join("..","Resources","Data","Table_B19083_2020","Table_B19083_2020_data_with_overlays.csv")


In [38]:
# Read CSV file into DataFrame
census_df = pd.read_csv(census_data_to_load,low_memory=False)
census_df

,FILEID,STUSAB,SUMLEV,GEOVAR,GEOCOMP,CHARITER,CIFSN_x,LOGRECNO,GEOID,GEOCODE,...,P0050001,P0050002,P0050003,P0050004,P0050005,P0050006,P0050007,P0050008,P0050009,P0050010
0,PLST,MO,50,0,0,0,0,2,0500000US29001,29001,...,2637,279,50,20,194,15,2358,2332,0,26
1,PLST,MO,50,0,0,0,0,3,0500000US29003,29003,...,129,129,0,0,129,0,0,0,0,0
2,PLST,MO,50,0,0,0,0,4,0500000US29005,29005,...,90,77,9,0,68,0,13,0,0,13
3,PLST,MO,50,0,0,0,0,5,0500000US29007,29007,...,1896,1489,1242,0,225,22,407,0,0,407
4,PLST,MO,50,0,0,0,0,6,0500000US29009,29009,...,236,236,44,0,192,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3098,PLST,MD,50,0,0,0,0,24,0500000US24047,24047,...,745,622,218,10,387,7,123,0,0,123
3099,PLST,MD,50,0,0,0,0,25,0500000US24510,24510,...,18001,5865,2568,122,3067,108,12136,10098,0,2038
3100,PLST,DE,50,0,0,0,0,2,0500000US10001,10001,...,4081,979,111,42,764,62,3102,2813,93,196
3101,PLST,DE,50,0,0,0,0,3,0500000US10003,10003,...,15899,6348,3390,72,2747,139,9551,8232,0,1319


In [39]:
#We are interested in the keeping columns relating to Population, Housing Count, 
# "NAME":"County Name"
# GEOID - code with last 5 digits equal to state fips + 0(0) + county fips
# "P0010001":"Total Population"
# "P0010003": "White alone"
# "P0010004": Black alone
# "P0010005": American Indian and Alaska Native Alone
# "P0010006": Asian Alone
# "P0010007": Native Hawaiian and Other Pacific Islander alone
# "P0010008":  Some other race alone
# "P0010009":"Population of 2 or more races"
# "P0020002":"Total Population: Hispanic Or Latino"
# "P0020003":"Total Population: Not Hispanic or Latino"

# See more: https://www2.census.gov/census_2010/01-Redistricting_File--PL_94-171/0FILE_STRUCTURE.pdf

col=["NAME","GEOID","GEOCODE","P0010001","P0010003","P0010004","P0010005","P0010006",
                "P0010007","P0010008","P0010009","P0020002","P0020003"]

df = census_df.loc[:,col]
df.head()

,NAME,GEOID,GEOCODE,P0010001,P0010003,P0010004,P0010005,P0010006,P0010007,P0010008,P0010009,P0020002,P0020003
0,Adair County,0500000US29001,29001,25314,21843,1279,51,687,13,230,1211,661,24653
1,Andrew County,0500000US29003,29003,18135,16907,156,53,89,1,82,847,391,17744
2,Atchison County,0500000US29005,29005,5305,5008,23,25,8,0,30,211,73,5232
3,Audrain County,0500000US29007,29007,24962,21648,1307,89,114,2,374,1428,731,24231
4,Barry County,0500000US29009,29009,34534,28655,106,453,811,73,1693,2743,3333,31201


In [40]:
# Create a function to add 1 to a number
def add_one(num):
    return(num+1)

In [41]:
# replace zero values in population columns with a 1 so Shannon index calculates without giving NaNs
df[["P0010004","P0010005","P0010006","P0010007","P0010008","P0010009"]] = df[["P0010004","P0010005","P0010006","P0010007","P0010008","P0010009"]].apply(add_one)

In [42]:
# Calculate non-white population
df["Non-White"] = df[["P0010004","P0010005","P0010006","P0010007","P0010008","P0010009"]].sum(axis=1)


In [43]:
# Rename the columns
# "P0010001":"Total Population"
# "P0010002" - Population of one race
# "P0010003": "White alone"
# "P0010004": Black alone
# "P0010005": American Indian and Alaska Native Alone
# "P0010006": Asian Alone
# "P0010007": Native Hawaiian and Other Pacific Islander alone
# "P0010008":  Some other race alone
# "P0010009":"Population of 2 or more races"
# "P0020002":"Total Population: Hispanic Or Latino"
# "P0020003":"Total Population: Not Hispanic or Latino"
# "P0020005": Total Population: Not Hispanic or Latino: White alone
# "P0020006":  Total Population: Not Hispanic or Latino: Black or African American alone
# "P0020007":  Total Population: Not Hispanic or Latino: American Indian and Alaska Native alone
# "P0020008":  Total Population: Not Hispanic or Latino: Asian alone
# "P0020009":  Total Population: Not Hispanic or Latino: Native Hawaiian and Other Pacific Islander alone 
# "P0020010": Total Population: Not Hispanic or Latino: Some other race alone
# "P0020011": Total Population: Not Hispanic or Latino: Population of two or more races
df.rename(columns={"NAME": "County",
                                        "P0010001":"Total Population",
                                        "P0010003":"White",
                                        "P0010004": "Black",
                                        "P0010005": "Native",
                                        "P0010006": "Asian",
                                        "P0010007": "Pacific Islander",
                                        "P0010008":"Other",
                                        "P0010009":"2+ Races",
                                        "P0020002":"Hispanic",
                                        "P0020003":"Not Hispanic",
                                        },inplace=True)


In [44]:
df.head()

,County,GEOID,GEOCODE,Total Population,White,Black,Native,Asian,Pacific Islander,Other,2+ Races,Hispanic,Not Hispanic,Non-White
0,Adair County,0500000US29001,29001,25314,21843,1280,52,688,14,231,1212,661,24653,3477
1,Andrew County,0500000US29003,29003,18135,16907,157,54,90,2,83,848,391,17744,1234
2,Atchison County,0500000US29005,29005,5305,5008,24,26,9,1,31,212,73,5232,303
3,Audrain County,0500000US29007,29007,24962,21648,1308,90,115,3,375,1429,731,24231,3320
4,Barry County,0500000US29009,29009,34534,28655,107,454,812,74,1694,2744,3333,31201,5885


In [45]:
# Add columns for percentage of each race/ethnicity
df["% White"] = df["White"]/df["Total Population"]
df["% Black"] = df["Black"]/df["Total Population"]
df["% Native"] = df["Native"]/df["Total Population"]
df["% Asian"] = df["Asian"]/df["Total Population"]
df["% Pacific Islander"] = df["Pacific Islander"]/df["Total Population"]
df["% Other"] = df["Other"]/df["Total Population"]
df["% Non-White"] = df["Non-White"]/df["Total Population"]
df["% Hispanic"] = df["Hispanic"]/df["Total Population"]
df["% Not Hispanic"] = df["Not Hispanic"]/df["Total Population"]
df["% 2+ Races"] = df["2+ Races"]/df["Total Population"]

In [46]:
# Simpson's diversity index = 1-(sum(n(n-1))/N(N-1))
df['Simpson Race DI'] = 1 - \
    (
        df['White']*(df['White']-1) + \
        df['Black']*(df['Black']-1) + \
        df['Native']*(df['Native']-1) + \
        df['Asian']*(df['Asian']-1) + \
        df['Pacific Islander']*(df['Pacific Islander']-1) + \
        df['Other']*(df['Other']-1) + \
        df['2+ Races'] * (df['2+ Races']-1)
    ) \
    /(df['Total Population']*(df['Total Population']-1))


In [47]:
df['Simpson Ethnic DI'] = 1 - \
        (
            df['Hispanic']*(df['Hispanic']-1) + \
            df['Not Hispanic']*(df['Not Hispanic']-1)
        ) \
        /(df['Total Population']*(df['Total Population']-1))


In [48]:
# Shannon diversity index = -1* sum((n/N) * ln(n/N))
df['Shannon Race DI']=(-1) * \
    (
        (df['% White'] * np.log(df['% White'])) + \
        (df['% Hispanic'] * np.log(df['% Hispanic'])) + \
        (df['% Black'] * np.log(df['% Black'])) + \
        (df['% Native'] * np.log(df['% Native'])) + \
        (df['% Asian'] * np.log(df['% Asian'])) + \
        (df['% Pacific Islander'] * np.log(df['% Pacific Islander'])) + \
        (df['% Other'] * np.log(df['% Other'])) + \
        (df['% 2+ Races'] * np.log(df['% 2+ Races']))
    )

In [49]:
# Shannon diversity index for Hispanic/Not Hispanic
df['Shannon Ethnic DI']=(-1) * \
    (
        (df['% Not Hispanic'] * np.log(df['% Not Hispanic'])) + \
        (df['% Hispanic'] * np.log(df['% Hispanic']))
    )

In [50]:
# show final dataframe
#oregon_counties_pop_df = oregon_counties_pop_df[["County","GEOCODE","Total Population","White","% White","Non-White","% Non-White","Hispanic","% Hispanic","Not Hispanic","% Not Hispanic"]]
df.sample(n=10)

,County,GEOID,GEOCODE,Total Population,White,Black,Native,Asian,Pacific Islander,Other,...,% Pacific Islander,% Other,% Non-White,% Hispanic,% Not Hispanic,% 2+ Races,Simpson Race DI,Simpson Ethnic DI,Shannon Race DI,Shannon Ethnic DI
1976,Prince William County,0500000US51153,51153,482204,200831,97458,3566,50315,601,66346,...,0.001246,0.137589,0.583527,0.252018,0.747982,0.130843,0.738699,0.377010,1.854763,0.564541
2489,Montgomery County,0500000US37123,37123,25751,16917,4227,140,394,10,2747,...,0.000388,0.106675,0.343288,0.152382,0.847618,0.051338,0.527219,0.258334,1.345847,0.426819
800,Fillmore County,0500000US27045,27045,21228,20278,102,17,72,3,162,...,0.000141,0.007631,0.045035,0.016064,0.983936,0.028265,0.086613,0.031613,0.299999,0.082296
1534,Wheeler County,0500000US48483,48483,4990,3748,87,58,28,3,519,...,0.000601,0.104008,0.250100,0.245892,0.754108,0.110822,0.412358,0.370932,1.195037,0.557777
853,Swift County,0500000US27151,27151,9838,8807,89,50,79,134,272,...,0.013621,0.027648,0.105408,0.067290,0.932710,0.041980,0.195748,0.125537,0.679680,0.246572
2540,Clay County,0500000US46027,46027,14967,12809,236,601,362,5,159,...,0.000334,0.010623,0.144585,0.030601,0.969399,0.053518,0.262173,0.059332,0.732140,0.136824
2544,Davison County,0500000US46035,46035,19956,17790,158,653,118,1,436,...,0.000050,0.021848,0.108839,0.045751,0.954249,0.040389,0.202030,0.087319,0.637741,0.185808
2605,Juab County,0500000US49023,49023,11786,10972,19,99,35,45,171,...,0.003818,0.014509,0.069574,0.048702,0.951298,0.038266,0.131600,0.092668,0.489140,0.194675
753,Putnam County,0500000US47141,47141,79854,67940,2189,708,1094,53,3294,...,0.000664,0.041250,0.149272,0.078343,0.921657,0.057380,0.270126,0.144413,0.836614,0.274704
3063,Pike County,0500000US01109,1109,33009,18275,12139,193,589,22,440,...,0.000666,0.013330,0.446545,0.027417,0.972583,0.041110,0.556044,0.053332,1.089358,0.125645


## Read Gini index data

In [51]:
gini_df = pd.read_csv(gini_file_to_load)

In [52]:
gini_df.head()

,Gini Index,Margin of Error!!Gini Index,GEOID,Geographic Area Name
0,0.4552,0.0326,0500000US01001,"Autauga County, Alabama"
1,0.4566,0.0119,0500000US01003,"Baldwin County, Alabama"
2,0.5047,0.0252,0500000US01005,"Barbour County, Alabama"
3,0.4500,0.0408,0500000US01007,"Bibb County, Alabama"
4,0.4685,0.0247,0500000US01009,"Blount County, Alabama"


In [53]:
gini_df.drop(columns=["Margin of Error!!Gini Index"],inplace=True)
gini_df

,Gini Index,GEOID,Geographic Area Name
0,0.4552,0500000US01001,"Autauga County, Alabama"
1,0.4566,0500000US01003,"Baldwin County, Alabama"
2,0.5047,0500000US01005,"Barbour County, Alabama"
3,0.4500,0500000US01007,"Bibb County, Alabama"
4,0.4685,0500000US01009,"Blount County, Alabama"
...,...,...,...
3216,0.4942,0500000US72145,"Vega Baja Municipio, Puerto Rico"
3217,0.4471,0500000US72147,"Vieques Municipio, Puerto Rico"
3218,0.5419,0500000US72149,"Villalba Municipio, Puerto Rico"
3219,0.4987,0500000US72151,"Yabucoa Municipio, Puerto Rico"


## Merge the dataframes

In [54]:
gini_df.shape

(3221, 3)

In [55]:
df.shape

(3103, 28)

In [56]:
#movies_with_ratings_df = pd.merge(movies_df, rating_counts, left_on = 'kaggle_id', right_index = True, how = 'left')
data_frame = pd.merge(df,gini_df,how='inner',on='GEOID')

In [57]:
data_frame.shape

(3103, 30)

In [58]:
data_frame

,County,GEOID,GEOCODE,Total Population,White,Black,Native,Asian,Pacific Islander,Other,...,% Non-White,% Hispanic,% Not Hispanic,% 2+ Races,Simpson Race DI,Simpson Ethnic DI,Shannon Race DI,Shannon Ethnic DI,Gini Index,Geographic Area Name
0,Adair County,0500000US29001,29001,25314,21843,1280,52,688,14,231,...,0.137355,0.026112,0.973888,0.047879,0.249769,0.050862,0.676567,0.120956,0.5398,"Adair County, Missouri"
1,Andrew County,0500000US29003,29003,18135,16907,157,54,90,2,83,...,0.068045,0.021561,0.978439,0.046760,0.128535,0.042194,0.401734,0.104052,0.4138,"Andrew County, Missouri"
2,Atchison County,0500000US29005,29005,5305,5008,24,26,9,1,31,...,0.057116,0.013761,0.986239,0.039962,0.107177,0.027148,0.335013,0.072643,0.4299,"Atchison County, Missouri"
3,Audrain County,0500000US29007,29007,24962,21648,1308,90,115,3,375,...,0.133002,0.029285,0.970715,0.057247,0.241625,0.056856,0.654415,0.132246,0.4439,"Audrain County, Missouri"
4,Barry County,0500000US29009,29009,34534,28655,107,454,812,74,1694,...,0.170412,0.096514,0.903486,0.079458,0.302044,0.174402,0.905812,0.317354,0.4703,"Barry County, Missouri"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3098,Worcester County,0500000US24047,24047,52460,41845,6237,137,754,9,841,...,0.202459,0.039611,0.960389,0.050381,0.346609,0.076086,0.856209,0.166706,0.4880,"Worcester County, Maryland"
3099,Baltimore city,0500000US24510,24510,585708,163026,338479,2313,21211,187,28047,...,0.721670,0.078413,0.921587,0.055405,0.581873,0.144529,1.322896,0.274876,0.5123,"Baltimore city, Maryland"
3100,Kent County,0500000US10001,10001,181851,107685,46999,1150,4430,126,5659,...,0.407872,0.076882,0.923118,0.086928,0.573395,0.141942,1.305087,0.271086,0.4191,"Kent County, Delaware"
3101,New Castle County,0500000US10003,10003,570719,314231,146545,2157,35201,176,28715,...,0.449423,0.111291,0.888709,0.076570,0.618709,0.197811,1.464587,0.349206,0.4607,"New Castle County, Delaware"


In [59]:
data_frame

,County,GEOID,GEOCODE,Total Population,White,Black,Native,Asian,Pacific Islander,Other,...,% Non-White,% Hispanic,% Not Hispanic,% 2+ Races,Simpson Race DI,Simpson Ethnic DI,Shannon Race DI,Shannon Ethnic DI,Gini Index,Geographic Area Name
0,Adair County,0500000US29001,29001,25314,21843,1280,52,688,14,231,...,0.137355,0.026112,0.973888,0.047879,0.249769,0.050862,0.676567,0.120956,0.5398,"Adair County, Missouri"
1,Andrew County,0500000US29003,29003,18135,16907,157,54,90,2,83,...,0.068045,0.021561,0.978439,0.046760,0.128535,0.042194,0.401734,0.104052,0.4138,"Andrew County, Missouri"
2,Atchison County,0500000US29005,29005,5305,5008,24,26,9,1,31,...,0.057116,0.013761,0.986239,0.039962,0.107177,0.027148,0.335013,0.072643,0.4299,"Atchison County, Missouri"
3,Audrain County,0500000US29007,29007,24962,21648,1308,90,115,3,375,...,0.133002,0.029285,0.970715,0.057247,0.241625,0.056856,0.654415,0.132246,0.4439,"Audrain County, Missouri"
4,Barry County,0500000US29009,29009,34534,28655,107,454,812,74,1694,...,0.170412,0.096514,0.903486,0.079458,0.302044,0.174402,0.905812,0.317354,0.4703,"Barry County, Missouri"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3098,Worcester County,0500000US24047,24047,52460,41845,6237,137,754,9,841,...,0.202459,0.039611,0.960389,0.050381,0.346609,0.076086,0.856209,0.166706,0.4880,"Worcester County, Maryland"
3099,Baltimore city,0500000US24510,24510,585708,163026,338479,2313,21211,187,28047,...,0.721670,0.078413,0.921587,0.055405,0.581873,0.144529,1.322896,0.274876,0.5123,"Baltimore city, Maryland"
3100,Kent County,0500000US10001,10001,181851,107685,46999,1150,4430,126,5659,...,0.407872,0.076882,0.923118,0.086928,0.573395,0.141942,1.305087,0.271086,0.4191,"Kent County, Delaware"
3101,New Castle County,0500000US10003,10003,570719,314231,146545,2157,35201,176,28715,...,0.449423,0.111291,0.888709,0.076570,0.618709,0.197811,1.464587,0.349206,0.4607,"New Castle County, Delaware"


In [60]:
data_frame.columns

Index(['County', 'GEOID', 'GEOCODE', 'Total Population', 'White', 'Black',
       'Native', 'Asian', 'Pacific Islander', 'Other', '2+ Races', 'Hispanic',
       'Not Hispanic', 'Non-White', '% White', '% Black', '% Native',
       '% Asian', '% Pacific Islander', '% Other', '% Non-White', '% Hispanic',
       '% Not Hispanic', '% 2+ Races', 'Simpson Race DI', 'Simpson Ethnic DI',
       'Shannon Race DI', 'Shannon Ethnic DI', 'Gini Index',
       'Geographic Area Name'],
      dtype='object')

In [61]:
column_names = ['GEOCODE','Geographic Area Name', 'County', 'GEOID', 'Total Population', 'White', '% White', 
       'Black', '% Black', 'Native', '% Native',
       'Asian', '% Asian', 'Pacific Islander', '% Pacific Islander', 
       'Other', '% Other', '2+ Races', '% 2+ Races'
       'Non-White', '% Non-White', 
       'Hispanic', '% Hispanic',
       'Not Hispanic', '% Not Hispanic', 
       'Simpson Race DI', 'Simpson Ethnic DI',
       'Shannon Race DI', 'Shannon Ethnic DI', 'Gini Index']

data_frame.reindex(columns=column_names)

,GEOCODE,Geographic Area Name,County,GEOID,Total Population,White,% White,Black,% Black,Native,...,% Non-White,Hispanic,% Hispanic,Not Hispanic,% Not Hispanic,Simpson Race DI,Simpson Ethnic DI,Shannon Race DI,Shannon Ethnic DI,Gini Index
0,29001,"Adair County, Missouri",Adair County,0500000US29001,25314,21843,0.862882,1280,0.050565,52,...,0.137355,661,0.026112,24653,0.973888,0.249769,0.050862,0.676567,0.120956,0.5398
1,29003,"Andrew County, Missouri",Andrew County,0500000US29003,18135,16907,0.932286,157,0.008657,54,...,0.068045,391,0.021561,17744,0.978439,0.128535,0.042194,0.401734,0.104052,0.4138
2,29005,"Atchison County, Missouri",Atchison County,0500000US29005,5305,5008,0.944015,24,0.004524,26,...,0.057116,73,0.013761,5232,0.986239,0.107177,0.027148,0.335013,0.072643,0.4299
3,29007,"Audrain County, Missouri",Audrain County,0500000US29007,24962,21648,0.867238,1308,0.052400,90,...,0.133002,731,0.029285,24231,0.970715,0.241625,0.056856,0.654415,0.132246,0.4439
4,29009,"Barry County, Missouri",Barry County,0500000US29009,34534,28655,0.829762,107,0.003098,454,...,0.170412,3333,0.096514,31201,0.903486,0.302044,0.174402,0.905812,0.317354,0.4703
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3098,24047,"Worcester County, Maryland",Worcester County,0500000US24047,52460,41845,0.797655,6237,0.118891,137,...,0.202459,2078,0.039611,50382,0.960389,0.346609,0.076086,0.856209,0.166706,0.4880
3099,24510,"Baltimore city, Maryland",Baltimore city,0500000US24510,585708,163026,0.278340,338479,0.577897,2313,...,0.721670,45927,0.078413,539781,0.921587,0.581873,0.144529,1.322896,0.274876,0.5123
3100,10001,"Kent County, Delaware",Kent County,0500000US10001,181851,107685,0.592161,46999,0.258448,1150,...,0.407872,13981,0.076882,167870,0.923118,0.573395,0.141942,1.305087,0.271086,0.4191
3101,10003,"New Castle County, Delaware",New Castle County,0500000US10003,570719,314231,0.550588,146545,0.256773,2157,...,0.449423,63516,0.111291,507203,0.888709,0.618709,0.197811,1.464587,0.349206,0.4607
